<a href="https://colab.research.google.com/github/tamnhu2913/Applied-Text-Mining-in-Python/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [ ]:
def date_sorter():
  regex1 = r'(\d{1,2}[/-]\d{1,2}[/-](\d{2}|\d{4})\b)' #04/20/2009; 04/20/09; 4/20/09; 4/3/09
  regex2 = r'(\d{1,2}\s[A-Z][a-z]+\s\d{4})' #20 Mar 2009; 20 March 2009
  regex3 = r'([A-Z][a-z]+\.?\s\d{2},?\s\d{4})' #Mar. 20, 2009; March 20, 2009; Mar. 20 2009; Mar 20 2009
  regex4 = r'([A-Z][a-z]+,?\s\d{4})' #Mar. 2009; March 2009
  regex5 = r'(\d{1,2}/\d{4})' #6/2008; 12/2009
  regex6 = r'([1|2]\d{3})' #2009

  regex = f'({regex1}|{regex2}|{regex3}|{regex4}|{regex5}|{regex6})'
  parsed_date = df.str.extractall(regex)[0].reset_index(drop=True)
  parsed_date = parsed_date.str.split(expand = True)

  parsed_date[0] = parsed_date[0].replace({'Since': None,'Age': None,'In': None,
                                           'Janaury': 'January', 'Decemeber': 'December'})
  parsed_date.loc[461, 1] = None

  for i in range(parsed_date.shape[1]):
    parsed_date[i] = parsed_date[i].fillna('')
  parsed_date = (parsed_date[0]+' '+parsed_date[1]+' '+parsed_date[2]).str.strip().str.replace(r'[.,]','', regex = True)

  date_formats = ['%m/%d/%y', '%m/%d/%Y', '%m-%d-%y', '%d %b %Y', '%d %B %Y',
                    '%b %d %Y', '%B %d %Y', '%b %Y', '%B %Y', '%m/%Y', '%Y']

  dts = pd.to_datetime(parsed_date, format=date_formats[0], errors='coerce')
  for fmt in date_formats[1:]:
      dts = dts.fillna(pd.to_datetime(parsed_date, format=fmt, errors='coerce'))

  return pd.Series(dts.sort_values().index)